In [1]:
import pandas as pd
import numpy as np

In [49]:
dpath = "../data/fullData.csv"
df=pd.read_csv(dpath,sep=',', header=0)
datadict = pd.read_csv('../data/dataDefs.csv',sep=',',header=0,index_col=0)

In [71]:
datadict = datadict.iloc[:33,0]

In [105]:
def getSummary(df):
    uniqueList = []
    typeList = []
    valueList = []
    
    for col in df.columns:
        uniques = len(df[col][df[col].notnull()].unique())
        uniqueList.append(uniques)
        if(uniques>100):
            values = "too many to calculate"
        else:
            values = df[col].value_counts().iloc[:10]
            values = zip(values.index,values)
        valueList.append(values)
        typeList.append(np.dtype(df[col]))
        
    uniqueSeries = pd.Series(uniqueList, index=df.columns)
    valueSeries = pd.Series(valueList, index=df.columns)
    typeSeries = pd.Series(typeList, index=df.columns)
    indexSeries = pd.Series(range(df.shape[1]), index=df.columns)
    
    #uniques
    summaryItems = [
        ('nulls', df.shape[0] - df.count()),
        ('distinct_count', uniqueSeries),
        ('top10 (value,freq)', valueSeries),
        ('dtype', typeSeries),
        ('indexNo',indexSeries)
    ]
    summaryDF = pd.DataFrame.from_items(summaryItems)
    print 'Rows,Columns',df.shape
    return summaryDF

In [108]:
dfsum = getSummary(df)
dfsum['desc']=datadict
dfsum.to_csv("../data/data_summary.csv")

Rows,Columns (65211, 33)


In [109]:
dfsum

,nulls,distinct_count,"top10 (value,freq)",dtype,indexNo,desc
user_id,0,2917,too many to calculate,int64,0,student id (numeric)
group_id,0,55,"[(2211, 7569), (2059, 6270), (2920, 5227), (35...",int64,1,school id (numeric)
school,0,55,"[(Virginia College of Osteopathic Medicine, 75...",object,2,school name (string)
course_id,0,1,"[(6220, 65211)]",int64,3,course id (numeric)
case_id,0,1,"[(226210, 65211)]",int64,4,case id (numeric)
case_name,0,1,"[((CORE 16 V2) 16. MSK: Trauma, 65211)]",object,5,case name (string)
case_creation_date,0,2917,too many to calculate,object,6,date of case creation
case_modification_date,0,2917,too many to calculate,object,7,date that case was modified
end_of_case,0,2,"[(1, 60579), (0, 4632)]",int64,8,"case completed (number, 0 = not finished, 1 = ..."
endofsession_date,4632,2634,too many to calculate,float64,9,timestamp when end_of_case was set to 1


Some data cleanliness issues: some users managed to click hyperlinks that supposedly weren't there.  These are flagged in "hl_flag".  Does hl_num include expert links?

In [115]:
pd.crosstab(df.hl_num,df.new_hl_clicked_num)

new_hl_clicked_num,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,19,20,22
hl_num,,,,,,,,,,,,,,,,,,,,,
0,25536,14,2,1,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,11058,5075,600,101,64,57,31,34,51,11,...,4,4,6,2,1,0,0,0,0,1
3,3771,596,380,624,150,36,22,5,5,3,...,0,1,1,0,0,0,0,0,0,0
4,4834,1652,480,651,553,165,64,29,10,5,...,3,0,0,1,0,2,1,1,0,0
5,1456,643,371,109,93,115,37,16,3,4,...,1,0,0,0,0,0,1,0,0,0
6,1375,225,220,175,168,152,253,134,66,29,...,8,8,2,3,1,1,0,0,0,0
8,1065,365,328,236,161,154,141,142,116,39,...,8,5,5,3,2,0,0,0,0,0


Shouldn't hl_clicked_num be -1 when hl_num is zero?  There are ~200 cases where this is not true.  Furthermore, hl_clicked_num should NOT be -1 when hl_num>0.

In [117]:
pd.crosstab(df.hl_num,df.hl_clicked_num)

hl_clicked_num,-1,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,19,20,22
hl_num,,,,,,,,,,,,,,,,,,,,,
0,25365,171,14,2,1,4,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,10981,77,5075,600,101,64,57,31,34,51,...,4,4,6,2,1,0,0,0,0,1
3,3751,20,596,380,624,150,36,22,5,5,...,0,1,1,0,0,0,0,0,0,0
4,4798,36,1652,480,651,553,165,64,29,10,...,3,0,0,1,0,2,1,1,0,0
5,1448,8,643,371,109,93,115,37,16,3,...,1,0,0,0,0,0,1,0,0,0
6,1366,9,225,220,175,168,152,253,134,66,...,8,8,2,3,1,1,0,0,0,0
8,1057,8,365,328,236,161,154,141,142,116,...,8,5,5,3,2,0,0,0,0,0


##Multimedia links

In [119]:
pd.crosstab(df.mm_num, df.magnify_clicked_num)

magnify_clicked_num,-1,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,17,19,20,21,23
mm_num,,,,,,,,,,,,,,,,,,,,,
0,25453,172,9,2,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10009,115,4096,1839,675,229,99,22,9,6,...,1,0,0,1,0,0,0,0,0,0
2,10149,107,732,3158,1090,767,391,189,113,54,...,10,5,1,1,1,0,1,1,0,0
3,3467,33,123,76,1450,211,153,49,24,15,...,2,3,0,0,2,1,0,0,1,1


In [120]:
pd.crosstab(df.mm_num, df.new_magnify_clicked_num)

new_magnify_clicked_num,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,17,19,20,21,23
mm_num,,,,,,,,,,,,,,,,,,,,,
0,25625,9,2,3,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10124,4096,1839,675,229,99,22,9,6,4,...,1,0,0,1,0,0,0,0,0,0
2,10256,732,3158,1090,767,391,189,113,54,49,...,10,5,1,1,1,0,1,1,0,0
3,3500,123,76,1450,211,153,49,24,15,10,...,2,3,0,0,2,1,0,0,1,1


##Expert link clicks
Note: number of expert links available is unknown.  Is there an expert link for every question?

In [124]:
pd.value_counts(df.expert_clicked_num)

-1    60224
 1     4513
 2      390
 3       67
 4        9
 6        5
 5        3
dtype: int64

##Answers
NOAD answertype means the card does not have a question on it.  Does -1 on NCAD and MCAD mean the respondent did not answer the question?

NOTE: only MCAD answer types are valid for measuring performance.  NCAD are uncorrected free text responses.

In [126]:
pd.crosstab(df.answertype,df.success)

success,-1,0,14,20,33,42,60,66,71,100
answertype,,,,,,,,,,
MCAD,58,2307,151,2912,1037,832,3538,1027,1086,4015
NCAD,51,0,0,0,0,0,0,0,0,2794
NOAD,45403,0,0,0,0,0,0,0,0,0


What about scoring?  What does hasbeenrated mean?

In [129]:
pd.crosstab(df.hasbeenrated,df.success)

success,-1,0,14,20,33,42,60,66,71,100
hasbeenrated,,,,,,,,,,
0,45458,0,0,0,0,0,0,0,0,0
1,54,2307,151,2912,1037,832,3538,1027,1086,6809


In [130]:
pd.crosstab(df.hasbeenrated,df.answertype)

answertype,MCAD,NCAD,NOAD
hasbeenrated,,,
0,58,5,45395
1,16905,2840,8


In [133]:
pd.crosstab(df.answertype[df.hasbeenrated==1],df.success[df.hasbeenrated==1])

success,-1,0,14,20,33,42,60,66,71,100
answertype,,,,,,,,,,
MCAD,0,2307,151,2912,1037,832,3538,1027,1086,4015
NCAD,46,0,0,0,0,0,0,0,0,2794
NOAD,8,0,0,0,0,0,0,0,0,0


In [134]:
pd.crosstab(df.hasbeenrated[df.answertype=='MCAD'],df.success[df.answertype=='MCAD'])

success,-1,0,14,20,33,42,60,66,71,100
hasbeenrated,,,,,,,,,,
0,58,0,0,0,0,0,0,0,0,0
1,0,2307,151,2912,1037,832,3538,1027,1086,4015


##Cards per session
A little bit of inconsistency: end_of_case should be true only if a user got through all 23 cards.  But there are 116 users who got through all 23 cards but were marked end_of_case=False; and 4 users who were marked end_of_case=True, but did not get through all 23 cards.  

In [188]:
grouped = df.groupby(['user_id','session_id'])
cards = pd.DataFrame(grouped.count()['card_id'])
cards['end_of_case'] = grouped.mean()['end_of_case']
cards['all_cards'] = cards.card_id==23

In [191]:
pd.crosstab(cards.all_cards,cards.end_of_case)

end_of_case,0.0,1.0
all_cards,,
False,166,4
True,116,2631


##Aggregating Data
Of the 23 cards in the course, 6 of them have multiple-choice questions. 
1. The first model should just aggregate these all together so that we can see on a user level, how much does engagement predict overall performance. 
1. From there we can look at engagement vs. performance for individual topics. 
1. Eventually I would like to see engagement on individual components - for example if there are 3 different links on a page, which of them contributes most to performance?
1. Lastly I would like to compare formative assessments to summative assessments. But summative assessment is not provided in this data set. 

When building model, must only use df.answertype=='MCAD' and df.hasbeenrated==1 when measuring performance. 
Need to also consider whether the student reached the last card. Most of the time students did in fact reach the last card.  

Aggregate to the user level.  There are 2917 users and 2917 sessions.
Engagement metrics to consider:
- binary variable for each card with links.  for example, card number six has a link.  did the user click it? then another variable for the next card with links
- really a variable for each resource (hl or mm/multimedia magnification) that was utilized
- handling time.  consider nonlinear options here.  either binning or decision tree
- expert clicked is tricky.  gotta figure out whether every multiple-choice answer has an expert

##TO DO:
Feature extraction:
1. Create performance score: unweighted average of scores on MCAD cards where hasbeenrated = 1.
1. Draw histogram of scores
1. create binary expert variable for every card where expert button is available: #5,9,14-17, 19 and 21
1. create binary hyperlink variable for every card where hyperlinks were available
1. create binary multimedia magnification variable for every card where mm was available

Model
1. build linear regression model
1. build decision tree model
1. translate to R code.

##Expert clicks
Looks like expert clicks are available on card #5,9,14-17, 19 and 21

In [194]:
pd.crosstab(df.card_num,df.new_expert_clicked_num)

new_expert_clicked_num,0,1,2,3,4,5,6
card_num,,,,,,,
1,2916,0,0,0,0,0,0
2,2906,0,0,0,0,0,0
3,2892,0,0,0,0,0,0
4,2883,1,0,0,0,0,0
5,2232,539,78,17,3,1,1
6,2863,0,0,0,0,0,0
7,2857,0,0,0,0,0,0
8,2855,0,0,0,0,0,0
9,2256,546,40,6,1,1,2


In [195]:
pd.crosstab(df.card_num,df.answertype)

answertype,MCAD,NCAD,NOAD
card_num,,,
1,0,0,2916
2,0,0,2906
3,0,0,2892
4,0,0,2884
5,2871,0,0
6,0,0,2863
7,0,0,2857
8,0,0,2855
9,2852,0,0
